In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os
import logging
import numpy as np
from hyperparams import hypers
from models.yolo import Yolov2Tiny
from models.loss import YoloLossMultiBoxes
from train import train_fn
from dataprocess.dataset import *
from dataprocess.dataprocess import *
from test import *
from utils import *

import yaml
from ultralytics import YOLO


In [2]:
import logging
# logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")
logging.basicConfig(level=logging.INFO, format="%(levelname)s - %(message)s")

In [3]:

if hypers.DATASET_PATH == "LOCAL":
    drive_dir=os.path.join(os.getcwd(), "archive","data")
elif hypers.DATASET_PATH == "DRIVE":
    drive_dir=os.path.join(os.getcwd(),"drive","MyDrive","archive","data")



#Dataframe Creation
train_labels=os.path.join(drive_dir,"train_solution_bounding_boxes (1).csv")
train_imgs_dir=os.path.join(drive_dir,"training_images")
test_imgs_dir=os.path.join(drive_dir,"testing_images")

labels_dir=os.path.join(drive_dir,"label")
if os.path.exists(labels_dir) == False:
  os.mkdir(labels_dir)

images_dir=os.path.join(drive_dir,"image")
if os.path.exists(images_dir) == False:
  os.mkdir(images_dir)

#YOLOV8 PART START

yolo_dir=os.path.join(drive_dir,"yolov8")
if os.path.exists(yolo_dir) == False:
  os.mkdir(yolo_dir)

labels_dir_v8=os.path.join(yolo_dir,"labels")
if os.path.exists(labels_dir_v8) == False:
  os.mkdir(labels_dir_v8)

if os.path.exists(labels_dir_v8+"/train") == False:
  os.mkdir(labels_dir_v8+"/train")
if os.path.exists(labels_dir_v8+"/val") == False:
  os.mkdir(labels_dir_v8+"/val2")

images_dir_v8=os.path.join(yolo_dir,"images")
if os.path.exists(images_dir_v8) == False:
  os.mkdir(images_dir_v8)

if os.path.exists(images_dir_v8+"/train") == False:
  os.mkdir(images_dir_v8+"/train")
if os.path.exists(images_dir_v8+"/val") == False:
  os.mkdir(images_dir_v8+"/val")

prediction_dir=os.path.join(yolo_dir,"predictions")
if os.path.exists(prediction_dir) == False:
  os.mkdir(prediction_dir)


imgs_list=list(sorted(os.listdir(train_imgs_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]


imgs_list=list(sorted(os.listdir(train_imgs_dir)))



In [4]:
if len(list(sorted(os.listdir(labels_dir)))) == 0:
    logging.info("Populating v1 directories")
    populate_v1_dir(drive_dir, imgs_list, images_dir, labels_dir, train_imgs_dir)
    logging.info("Populated v1 directories")
    
if len(list(sorted(os.listdir(labels_dir_v8+"/train")))) == 0:
    logging.info("Populating v8 directories")
    populate_v8_dir(drive_dir, imgs_list, images_dir_v8, labels_dir_v8, train_imgs_dir, val_idx)
    logging.info("Populated v1 directories")
    
yaml_path = os.path.join(yolo_dir,"yolo.yaml")
if os.path.exists(yaml_path) == False:
    generate_yolov8_yaml(yolo_dir, images_dir_v8)

In [5]:
if hypers.LOAD_MODEL_v8:
    model = YOLO(hypers.path_best_weights_v8)
else:
    model=YOLO('yolov8m.pt')
# model.train(data=yolo_dir+'/yolo.yaml',epochs=20,patience=5,batch=8,
#                     lr0=0.0005,imgsz=640)

In [17]:
# # metrics = model.val()
# results, test_img_list = predict_yolo_v8(model, test_imgs_dir, prediction_dir)

# imgs_name=['vid_5_400','vid_5_26720']
# save_sample_pics(test_imgs_dir, imgs_name)

In [6]:
model = Yolov2Tiny(split_size=7, num_boxes=hypers.NUM_BOXES, num_classes=hypers.NUM_CLASSES).to(hypers.DEVICE)
optimizer = optim.Adam( model.parameters(), lr=hypers.LEARNING_RATE, weight_decay=hypers.WEIGHT_DECAY )
loss_fn = YoloLossMultiBoxes(S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES)

In [7]:
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

annotations=create_annotations(images_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt
annotations_predict=create_annotations(test_imgs_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt


dataset=VOCDataset(annotations, labels_dir, images_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)
dataset_predict=VOCDatasetPredict(annotations_predict, test_imgs_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)


train_set, test_set = torch.utils.data.random_split(dataset, [300, 55])

train_loader = DataLoader(dataset=train_set, batch_size=hypers.BATCH_SIZE,
                            pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

test_loader = DataLoader(dataset=test_set, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)


predict_loader = DataLoader(dataset=dataset_predict, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

In [ ]:
map_array_train = []
map_array_test = []
for epoch in tqdm(range(10), position=0, leave=True):
    
    pred_boxes, target_boxes = get_bboxes(train_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
    mean_avg_prec_train = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
    pred_boxes, target_boxes = get_bboxes(test_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
    mean_avg_prec_test = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
    map_array_train.append(mean_avg_prec_train.item())
    map_array_train.append(mean_avg_prec_test.item())
    
    print(f"epoch: {epoch} Train mAP: {mean_avg_prec_train}")
    print(f"epoch: {epoch} Test mAP: {mean_avg_prec_test}")
    
    # k=0
    # if (mean_avg_prec > 0.9) and (k==0):
    #     k=1
    #     checkpoint = {
    #         "state_dict": model.state_dict(),
    #         "optimizer": optimizer.state_dict(),
    #     }
    #     save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE, exit_training=True)
    #     # time.sleep(10)

    train_fn(train_loader, model, optimizer, loss_fn)
    


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 Train mAP: 0.0
epoch: 0 Test mAP: 0.0


 10%|█         | 1/10 [00:10<01:38, 10.92s/it]

Mean loss was 142.1671002705892
epoch: 1 Train mAP: 0.0
epoch: 1 Test mAP: 0.0


 20%|██        | 2/10 [00:21<01:26, 10.83s/it]

...

 30%|███       | 3/10 [00:32<01:15, 10.75s/it]

Mean loss was 30.682282235887314
epoch: 3 Train mAP: 0.0024957258719950914
epoch: 3 Test mAP: 0.00021404089056886733


 40%|████      | 4/10 [00:43<01:04, 10.82s/it]

Mean loss was 22.000303586324055
epoch: 4 Train mAP: 0.6103190779685974
epoch: 4 Test mAP: 0.043168842792510986


 33%|███▎      | 6/18 [00:01<00:03,  3.49it/s, loss=15.8]

In [ ]:
print(f"mAP Array: {map_array}")

In [ ]:
# index = 0

    # for x, y in train_loader:
    #     x = x.to(DEVICE)
    #     index += 1
    #     for idx in range(8):
    #         if index < 2:
    #             continue
    #         bboxes = cellboxes_to_boxes(model(x), S=7, B=2, C=1)
    #         bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
    #         plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

    #     # import sys
    #     # sys.exit()

    #     if index == 3:
    #         break